# PHYS20762 Computational Physics
## Project 3: Monte Carlo Techniques
### Neutron Transport and Scattering Through a Shielding Layer

Author: David Phelan  
The University of Manchester  
Date: May 2025  

---

## 1. Introduction

In this project, we use Monte Carlo techniques to simulate the behaviour of thermal neutrons passing through various shielding materials. The aim is to model neutron scattering, absorption, and transmission processes within slabs of water, lead, and graphite.

By generating random numbers, random points in three dimensions, and implementing exponential distributions, we build up to a full neutron transport simulation. We will visualise random walks, quantify transmission, reflection, and absorption rates, and determine the characteristic attenuation lengths for different materials.

### 1.1 Objectives

  1. Verify the generation of uniform random numbers using `numpy.random.uniform()`.
  1. Generate and visualise random 3D points to assess distribution properties.
  1. Create and test an exponential random number generator.
  1. Produce isotropic random directions and isotropic steps with exponentially distributed lengths.
  1. Simulate neutron random walks through slabs of different materials.
  1. Quantify neutron absorption, reflection, and transmission rates as functions of slab thickness.
  1. Determine attenuation lengths from transmission data.
  1. Implement the Woodcock method to simulate transport through two adjacent materials.

Throughout the project, we aim to support the simulation with appropriate plots and discuss the numerical results, including any uncertainties.

## 2. Setup & Initialisation

1. Using pip to install essential libraries
1. Importing the required libraries, functions and types

In [70]:
%pip install numpy scipy plotly nbformat pandas

Note: you may need to restart the kernel to use updated packages.


In [71]:
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from collections import Counter
from IPython.display import display, Markdown
import textwrap
import time
import math

### Random Seed for Reproducibility
To ensure consistent and reproducible results across different runs of the notebook, we set a fixed random seed at the start of the script. This guarantees that each random number generated will follow the same sequence every time the notebook is executed (assuming each cell is executed once in order).

In [72]:
np.random.seed(42)
start = time.time()

### Helpers

`print_markdown` is a small helper function to print markdown. It's used for displaying results at various points during the project.

`textwrap.dedent` is used to support the case where you use triple quoted stings over multiple lines indented, for example:

```python
def print_list():
    print_markdown(f"""
    - Point 1
    - Point 2
    """)
```

would output:

```markdown
- Point 1
- Point 2
```

without `textwrap.dedent`, and by including it we get the expected output:

- Point 1
- Point 2

In [73]:
def print_markdown(markdown):
    display(Markdown(textwrap.dedent(markdown)))

`plot` provides a consistent rendering experience that can be adjusted to suit the user's display. If the figures are too large or small in one dimension for your monitor, please adjust the values here.

In [74]:
def plot(figure):
    figure.update_layout(
        autosize=True,
        width=1920/2,
        height=1080/2,
    )
    figure.show()
    return figure

`format_appropriate` is used to format numbers to an appropriate precision, given their error

In [75]:

def format_appropriate(value, error, sig=2):
    def round_sig(x, sig=2):
        if x == 0:
            return 0.0
        else:
            return round(x, sig - int(math.floor(math.log10(abs(x)))) - 1)

    def format_sig(x, sig=2):
        if x == 0:
            return f"{0:.{sig - 1}f}"
        else:
            digits = sig - int(math.floor(math.log10(abs(x)))) - 1
            return f"{x:.{max(digits, 0)}f}"

    rounded_error = round_sig(error, sig)
    if rounded_error == 0:
        return f"{value}", f"{rounded_error}"

    decimal_places = -int(math.floor(math.log10(abs(rounded_error)))) + (sig - 1)
    rounded_value = round(value, decimal_places)
    return f"${rounded_value:.{decimal_places}f} \\pm {format_sig(rounded_error, sig)}$"



## 3. Simulation Classes

To manage the material properties and slab geometries systematically, we define two classes: `Material` and `Slab`.

### `Material` Class

The `Material` class encapsulates the physical properties needed to simulate neutron interactions, including:

- **Scattering cross-section** ($σ_s$)
- **Absorption cross-section** ($σ_a$)
- **Total cross-section** (sum of scattering and absorption)
- **Absorption probability**

It also includes a `construct_from_microscopic_properties` method to conveniently calculate macroscopic cross-sections based on:

- `density` $\rho$
- `molar_mass` $M$
- Microscopic cross-sections (given in barns, converted to ${cm}^2$)

This design improves clarity and avoids recalculating these properties throughout the notebook.

### `Slab` Class

The `Slab` class represents a slab of shielding material, characterised by:

- A `Material` object (e.g., water, lead, graphite)
- A `width` (thickness $L$ through which neutrons travel

By creating a separate `Slab` class, we make the simulation modular, allowing us to easily vary material type and slab thickness during experiments.

In [76]:
class Material:
    BARN_TO_CM2 = 1e-24  # cm²
    AVOGADRO_CONSTANT = 6.02214076e23  # 1/mol

    def __init__(self, name, scattering_cross_section, absorption_cross_section):
        self.name = name
        self.scattering_cross_section = scattering_cross_section
        self.absorption_cross_section = absorption_cross_section
        self.total_cross_section = scattering_cross_section + absorption_cross_section
        self.absorption_probability = absorption_cross_section / self.total_cross_section if self.total_cross_section > 0 else 0

    @staticmethod
    def construct_from_microscopic_properties(name, sigma_a, sigma_s, density, molar_mass):
        number_density = Material.AVOGADRO_CONSTANT * density / molar_mass
        

        return Material(
            name,
            number_density * sigma_s * Material.BARN_TO_CM2,
            number_density * sigma_a * Material.BARN_TO_CM2
        )
    
class Slab:
    def __init__(self, material, width):
        self.material = material
        self.width = width

### Initialisation of Materials

We instantiate the following materials using their known physical properties:

In [77]:
# Define materials with their properties
water = Material.construct_from_microscopic_properties(
    "Water", 0.6652, 103.0, 1.0, 18.0153)

lead = Material.construct_from_microscopic_properties(
    "Lead", 0.158, 11.221, 11.35, 207.2)

graphite = Material.construct_from_microscopic_properties(
    "Graphite", 0.0045, 4.74, 1.67, 12.011)

materials = [water, lead, graphite]

Each material's macroscopic properties are now readily accessible for subsequent neutron transport simulations.

## 4. Testing Random Number Generators

### 4.1 Uniform Random Number Generator (`numpy.random.uniform`)

We first test Python's built-in uniform random number generator, `numpy.random.uniform()`. 
In one dimension, a uniform random generator should produce a flat probability density between 0 and 1. 
In three dimensions, random (x, y, z) points should fill the space evenly without any clustering, streaks, or artefacts. 
We visualise the results using a histogram of random values and a 3D scatter plot.

In [78]:
def test_random_number_generator():
    x = np.random.uniform(0, 1, 1000)
    y = np.random.uniform(0, 1, 1000)
    z = np.random.uniform(0, 1, 1000)

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Uniform)", "3D Scatter (Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Uniform 3D Points"
        ),
        row=1, col=2
    )

    plot(fig)

test_random_number_generator()

### 4.2 Biased Random Number Generator (`RandSSP`)

Compare the results above to the same plots using a flawed random number generator, `RandSSP`, which is known to introduce bias into random samples.
Such flaws can lead to visible structure in the generated points, such as banding or clustering, which should not occur for truly random distributions.

In [79]:
class RandSSP:
    def __init__(self, seed=123456789):
        self.m = 2**31
        self.a = 2**16 + 3
        self.c = 0
        self.x = seed

    def generate(self, p, q):
        r = np.zeros((p, q))
        for l in range(q):
            for k in range(p):
                self.x = (self.a * self.x + self.c) % self.m
                r[k, l] = self.x / self.m
        return r

In [80]:
def test_random_number_generator():
    r = RandSSP().generate(3, 1500)
    x, y, z = r[0, :], r[1, :], r[2, :]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Non-Uniform)", "3D Scatter (Non-Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Non-Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Non-Uniform 3D Points"
        ),
        row=1, col=2
    )

    plot(fig)

test_random_number_generator()

The `RandSSP` generator produces clear evidence of bias. With the 2D histogram it is difficult to see the bias, but plotting the values in 3 dimensions reveals points aligned in planes, highlighting structural artefacts.

## 5 Survival Function Fitting

To determine the attenuation length of neutrons passing through a shielding material, we analyse the *survival function* — the number of neutrons that have travelled at least a given distance without being absorbed.

A common approach is to bin neutron travel distances into discrete intervals and count the number of surviving neutrons per bin. However, binning introduces systematic errors whenever a continuous distribution is discretised. This is particularly problematic for exponential decay, which is highly left-skewed: many neutrons are absorbed at short distances, and relatively few travel far. As a result, binning can distort the survival curve, leading to bias in attenuation length estimates — especially when using bin midpoints or edges that misrepresent the average behaviour within each bin.

To avoid these artefacts, we instead adopt a *ranked method*. We generate continuous travel distances for a large number of neutrons, sort them in ascending order, and assign each a survival rank: the number of neutrons that travelled at least that far. This produces a smooth, high-resolution survival curve without requiring any binning. Taking the natural logarithm of the survival rank transforms the exponential decay law into a linear relationship of the form:


$$\log(N(x)) = \log(N_0) - \frac{x}{\lambda}$$


We fit this relation using weighted linear regression, with each point weighted by the square root of its rank. While these weights are not derived from an explicit statistical uncertainty model, they give greater influence to regions of the curve with more surviving neutrons, where statistical fluctuations would be relatively smaller in a binned context.

Each plot also includes a dashed line at $N_0 / e$, where $N_0$ is the initial number of neutrons. According to the exponential decay law, the distance at which the survival function intersects this line corresponds to the attenuation length $\lambda$ of the material.

This ranked, weighted approach allows us to obtain a precise estimate of attenuation length without introducing bias from binning. It is applied below to water, using the material’s absorption cross section and neglecting scattering, which is considered later via full random walk simulations.

In [81]:
def produce_exponential_distribution(samples, material):
    uniform_random_numbers = np.random.uniform(0, 1, samples)
    exponential_random_numbers = - np.log(uniform_random_numbers) / material.absorption_cross_section

    sorted_data = np.sort(exponential_random_numbers)
    ranks = np.arange(len(sorted_data), 0, -1)

    x_ranked = sorted_data
    y_ranked = np.log(ranks)

    weights_ranked = np.sqrt(ranks)

    fit_ranked, _ = np.polyfit(x_ranked, y_ranked, 1, w=weights_ranked, cov=True)
    fit_coeff, fit_intercept = fit_ranked

    return x_ranked, ranks, fit_coeff, fit_intercept

In [82]:
def plot_exponential_distribution(x_ranked, ranks, fit_coeff, fit_intercept, material, samples):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_ranked,
        y=ranks,
        mode='markers',
        name=f"Ranked survival ({material.name})"
    ))

    fitted_y = np.exp(fit_intercept + fit_coeff * x_ranked)
    fig.add_trace(go.Scatter(
        x=x_ranked,
        y=fitted_y,
        mode='lines',
        name=f"Exponential fit ({material.name})"
    ))

    samples_after_one_attenuation_length = samples / np.e

    fig.add_trace(go.Scatter(
        x=[0, max(x_ranked)],
        y=[samples_after_one_attenuation_length] * 2,
        mode='lines',
        line=dict(color='red', dash='dash'),
        name=f"{material.name}: N₀/e Line"
    ))

    fig.add_annotation(
        x=max(x_ranked) * 0.8,
        y=samples_after_one_attenuation_length,
        text="N₀/e",
        showarrow=False,
        font=dict(size=12, color="red"),
        yshift=10
    )

    fig.update_layout(
        title=f"Ranked Survival Function and Weighted Fit for {material.name}",
        xaxis_title="Distance (cm)",
        yaxis_title="Number of neutrons surviving",
        legend=dict(x=0.68, y=0.98)
    )

    plot(fig)

In [83]:
def calculate_reduced_chi_square(observed, expected):

    observed = np.asarray(observed)
    expected = np.asarray(expected)

    chi_squared = np.sum((observed - expected) ** 2 / expected)
    dof = len(observed) - 2
    reduced_chi_squared = chi_squared / dof

    return reduced_chi_squared


In [84]:
def print_results(samples):
    water_ranked_data = produce_exponential_distribution(samples=samples, material=water)
    water_x_ranked, water_ranks, water_fit_coeff, water_fit_intercept = water_ranked_data
    plot_exponential_distribution(
        water_x_ranked,
        water_ranks,
        water_fit_coeff,
        water_fit_intercept,
        material=water,
        samples=samples)

    data = [(water, water_ranked_data), *[
        (material, produce_exponential_distribution(samples=samples, material=material))
        for material in [lead, graphite]
    ]]

    markdown = "Material | Theoretical Attenuation Length (cm) | Simulated Attenuation Length (cm) | \n"
    markdown += " --- | --- | --- \n"
    for material, ranked_data in data:
        _, _, coeff, _ = ranked_data
        theoretical_attenuation_length = 1 / material.absorption_cross_section
        simulated_attenuation_length = -1 / coeff
        statistical_error = theoretical_attenuation_length / np.sqrt(samples)
        result = format_appropriate(simulated_attenuation_length, statistical_error, 2)
        markdown += f"{material.name} | ${theoretical_attenuation_length:.2f}$ |  {result} \n"

    print_markdown(markdown)

print_results(samples=10000)


Material | Theoretical Attenuation Length (cm) | Simulated Attenuation Length (cm) | 
 --- | --- | --- 
Water | $44.97$ |  $45.12 \pm 0.45$ 
Lead | $191.86$ |  $189.1 \pm 1.9$ 
Graphite | $2653.99$ |  $2643 \pm 27$ 


The results agree with the theoretical value, within the statistical error: $\frac{\lambda}{\sqrt{N_0}}$ which dominates for low samples ($N_0$) over the fitting uncertainty.

## 6. Generating Isotropic Directions

### 6.1 Generating Isotropic Unit Vectors

To simulate random neutron directions uniformly distributed over a sphere, we generate isotropic unit vectors $\vec{r} = x\hat{i} + y\hat{j} + z\hat{k}$ satisfying $|\vec{r}| = 1$. 
We use spherical polar coordinates $(\theta, \phi)$ where:
- $\theta$ is the polar angle, drawn from $\cos\theta \sim \text{uniform}(-1, 1)$,
- $\phi$ is the azimuthal angle, drawn uniformly from $[0, 2\pi)$.

This ensures points are uniformly distributed over the sphere's surface, representing isotropic scattering accurately.

If we sampled $\theta$ uniformly from $[0, \pi]$, the resulting points would cluster near the poles due to the non-uniform area element in spherical coordinates. By sampling $\cos\theta$ uniformly instead, we correct for this bias and ensure an even distribution over the sphere’s surface.


In [85]:
def generate_random_unit_vector():

    theta = np.arccos(np.random.uniform(-1, 1))
    phi = np.random.uniform(0, 2*np.pi)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    return x, y, z

## 6.1.1 Visualising the Isotropic Unit Vectors

To verify the uniformity of the generated directions, we visualise many unit vectors in 3D space. 
A truly isotropic distribution should appear as a uniform scatter of points across the surface of a sphere.

In [86]:
def visualize_random_unit_vectors():
    x,y,z = zip(*[(x,y,z) for x,y,z in [generate_random_unit_vector() for _ in range(10000)]])

    fig = px.scatter_3d(
        x=x, y=y, z=z, color=z,
        labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
    )
    fig.update_traces(marker=dict(size=2))  # Smaller points
    fig.update_layout(
        title="3D Scatter Plot"
    )
    plot(fig)

visualize_random_unit_vectors()

## 6.2 Simulating Random Walks in 3D Space

Using the isotropic unit vectors generated above, we now simulate random neutron paths through 3D space. 
At each step, a neutron travels in a random direction, with a step length sampled from an exponential distribution:

$$P(x) = \frac{1}{\lambda} e^{-x/\lambda}$$

where $\lambda$ is the mean free path determined by the material's macroscopic cross-section. 
This simulates the stochastic behaviour of neutrons undergoing scattering in matter.

Note: at this stage we are ignoring absorption, and just focusing on the scattering of the neutrons for a set number of paths.

In [87]:
def random_uniform_path_length(cross_section):
    return -np.log(np.random.uniform(0, 1)) / cross_section

In [88]:
def random_walk(cross_section):
    x, y, z = (0, 0, 0)
    yield (x, y, z)
    while True:
        step_size = random_uniform_path_length(cross_section)
        dx, dy, dz = generate_random_unit_vector()
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        yield (x, y, z)


## 6.2.1 Visualising Random Walks

We now simulate neutron random walks through different materials. 
Each trajectory represents the stochastic motion of a neutron, with direction sampled isotropically and step lengths sampled from the material's exponential attenuation law.

In [89]:
def visualise_random_walk(materials):
    for material in materials:
        macroscopic_cross_section= material.scattering_cross_section
        
        positions_iterator = random_walk(macroscopic_cross_section)
        positions = [next(positions_iterator) for _ in range(100)]

        x, y, z = zip(*positions)
        scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', marker=dict(size=2))
        layout = go.Layout(
            title=f"Random Walk in {material.name}",
            scene=dict(
                xaxis=dict(title='X'),
                yaxis=dict(title='Y'),
                zaxis=dict(title='Z')
            )
        )
        fig = go.Figure(data=[scatter], layout=layout)
        plot(fig)

visualise_random_walk(materials)

## 6.3 Visualising Neutron Random Walks in a Slab

We now simulate neutron random walks through a finite slab of material. 
Each slab is placed between two planes located at $x = 0$ and $x = L$, where $L$ is the slab thickness. 

Initially, neutrons are fired **perpendicularly into the slab** from the left boundary at $x = 0$, moving along the positive $x$-axis. 
Each neutron then undergoes a stochastic random walk within the slab:
- The **step size** along its path is drawn from an exponential distribution:
  
  $$P(x) = \frac{1}{\lambda} e^{-x/\lambda}$$
  
  where $\lambda$ is the mean free path based on the material's macroscopic cross-section.
- After each step, the neutron's new position is updated by moving along a randomly generated isotropic unit vector.

At each step, we determine whether the neutron:
- **Is absorbed** within the slab (random survival check against absorption probability),
- **Scatters** and continues walking within the slab,
- **Escapes** by crossing beyond the slab boundaries (either $x < 0$ or $x > L$).

The random walk terminates if the neutron is absorbed or escapes.
We visualise individual neutron trajectories through the slab for different materials, illustrating the stochastic scattering and absorption behaviour in each case.

The simple `random_walk` function is now replaced with a version that takes a slab of material and performs a full walk until an end condition (escape or absorb) is met.

In [90]:
def random_walk(slab):
    result = []
    x, y, z = (0, 0, 0)
    result.append((x, y, z))
    dx, dy, dz = (1, 0, 0)  # Initial direction
    while True:
        step_size = random_uniform_path_length(slab.material.total_cross_section)
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        if x > slab.width:
            return "Transmitted", result
        elif x < 0:
            return "Reflected", result
        elif np.random.uniform(0, 1) < slab.material.absorption_probability:
            return "Absorbed", result
        else:
            # Scattering occurs
            dx, dy, dz = generate_random_unit_vector()
        result.append((x, y, z))

`visualise_random_walks` plots the requested number of random walks through a given slab of material, colour coding them with the fate of the walk (Transmitted, Reflected or Absorbed).

Note: each walk is written on the plot as a separate 3D scatter so some adjustments are made to reduce clutter in the legend to only show a record for the first walk of each fate.

In [91]:
def visualise_random_walks(slab, number_of_neutrons):
    walks = [ random_walk(slab) for _ in range(number_of_neutrons) ]

    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    show_legend = {
        "Transmitted": True,
        "Reflected": True,
        "Absorbed": True
    }
    colours = {
        "Transmitted": "blue",
        "Reflected": "red",
        "Absorbed": "green"
    }
    fate_counts = Counter([walk[0] for walk in walks])

    def walk_to_scatter(walk):
        x, y, z = zip(*walk[1])
        showlegend = show_legend[walk[0]]
        show_legend[walk[0]] = False #only show the legend for the first walk of each type
        return go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers+lines',
            marker=dict(size=2),
            name=f'{walk[0]} ({fate_counts[walk[0]]})',
            line=dict(color=colours[walk[0]]),
            showlegend=showlegend)
    
    fig = go.Figure(data=[walk_to_scatter(walk) for walk in walks], layout=layout)
    fig.update_layout(
        title=f"Random Walk in {slab.material.name}"
    )
    plot(fig)

visualise_random_walks(Slab(water, 4), 30)


### 6.4 Quantifying Neutron Scattering, Absorption, and Transmission

We now quantify the outcomes of neutron interactions with the slab by simulating large numbers of random walks.

For a given slab thickness $L$, we fire $N$ neutrons perpendicularly into the slab at $x = 0$, each initially travelling along the positive $x$-direction. Each neutron undergoes a random walk based on the material's macroscopic cross-section, continuing until it is either:
- **Absorbed** within the slab,
- **Reflected** back through the entrance at $x < 0$,
- **Transmitted** through the far side at $x > L$.

We record the number of neutrons absorbed ($N_A$), reflected ($N_R$), and transmitted ($N_T$), and compute the corresponding rates:

$$\text{Absorption Rate} = \frac{N_A}{N}, \quad \text{Reflection Rate} = \frac{N_R}{N}, \quad \text{Transmission Rate} = \frac{N_T}{N}$$

The statistical uncertainty in each measured rate is estimated assuming binomial statistics:

$$\sigma = \sqrt{\frac{p(1-p)}{N}}$$

where $p$ is the measured probability of absorption, reflection, or transmission.

In [92]:

def binomial_standard_error(probability, trials):
    return np.sqrt(probability * (1 - probability) / trials)

def random_walks(slab, counts):
    return [ random_walk(slab) for _ in range(counts)]

def count_fates(slab, counts):
    return Counter([walk[0] for walk in random_walks(slab, counts)])

def quantify_random_walk(slab, number_of_neutrons):
    fate_counts = count_fates(slab, number_of_neutrons)
    for fate in fate_counts:
        probability = fate_counts[fate] / number_of_neutrons
        error = binomial_standard_error(probability, number_of_neutrons)
        yield fate, probability, error

In [93]:

def show_quantified_random_walk(slabs, number_of_neutrons):
    fig = go.Figure()
    for slab in slabs:
        fates = quantify_random_walk(slab, number_of_neutrons)
        fates = list(fates)
        fates = sorted(fates, key=lambda x: x[0])
        fates = [(fate[0], fate[1] * 100, fate[2] * 100) for fate in fates]
        fates = [(fate[0], fate[1], fate[2]) for fate in fates if fate[1] > 0.01]

        fig.add_trace(go.Bar(
            x=[fate[0] for fate in fates],
            y=[fate[1] for fate in fates],
            name=f"{slab.material.name} ({slab.width} cm)",
            error_y=dict(type='data', array=[fate[2] for fate in fates])
        ))

    fig.update_layout(
        title="Quantified Random Walk Results",
        xaxis_title="Fate",
        yaxis_title="Rate (%)",
        barmode='group',
        legend=dict(x=0.68, y=0.98),
        yaxis=dict(range=[0, 100])
    )
    plot(fig)

show_quantified_random_walk([Slab(water, 10), Slab(graphite, 10), Slab(lead, 10)], 1000)

## 6.4.3 Variation of Neutron Interaction Rates with Slab Thickness

To analyse how neutron behaviour depends on material thickness, we repeat the random walk simulation for a range of slab thicknesses $L$.

At each thickness, we compute the absorption, reflection, and transmission rates (with associated uncertainties) by simulating a large number of neutrons and recording their fates. 
We then plot the variation of these rates as functions of slab thickness for each material.

This part of the script is the most computationally expensive. If your computer is struggling to run the simuation, consider reducing the supplied trials parameter from 1000 to a lower value, however be aware the results will be less accurate.

In [94]:
def plot_width_vs_fates(trials, steps, materials):
    for material, max_width in materials:
        widths = np.linspace(0.000001, max_width, steps)
        data = { }
        for width in widths:
            fates = count_fates(Slab(material, width), trials)
            for fate, count in fates.items():
                probability = count / trials
                error = trials * binomial_standard_error(probability, trials)
                if fate not in data:
                    data[fate] = []
                
                data[fate].append((width, count, error))
       
        fig = go.Figure()

        def exponential_func(x, attenuation_length):
             return trials * np.exp(-x / attenuation_length)
        
        transmitted_widths, transmitted_counts, transmitted_errors = zip(*data["Transmitted"])
        popt, pcov = curve_fit(exponential_func, transmitted_widths, transmitted_counts)

        attenuation_length = popt[0]
        attenuation_length_error = np.sqrt(pcov[0][0])

        theoretical_x = np.linspace(0, max_width, 1000)
        theoretical_y_curve_fit = exponential_func(theoretical_x, attenuation_length)
        theoretical_trace_curve_fit = go.Scatter(
            x=theoretical_x,
            y=theoretical_y_curve_fit,
            mode='lines',
            name=f"Fitted survival function, {material.name}"
        )
        
        fig.add_trace(theoretical_trace_curve_fit)

        for fate_name in data:
            widths, counts, errors = zip(*data[fate_name])
            fig.add_trace(go.Scatter(
                x=widths,
                y=counts,
                mode='markers',
                name=f"{material.name} {fate_name}",
                error_y=dict(
                    type='data',
                    array=errors,
                    visible=True
                )
            ))

        fig.add_trace(go.Scatter(
            x=[attenuation_length, attenuation_length],
            y=[0, trials],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False
        ))

        fig.add_annotation(
            x=attenuation_length,
            y=trials * 0.8,
            text=f"Attenuation length: {attenuation_length:.2f} ± {attenuation_length_error:.2f}",
            showarrow=True,
            arrowhead=2,
            ax=-110,
            ay=-40
        )

        fig.update_layout(
            title=f"Survivability for various widths of {material.name}",
            xaxis_title="Width (cm)",
            yaxis_title="Surviving neutrons",
            legend=dict(
                x=1.05,
                y=1,
                xanchor="left",
                yanchor="top"
            ),
            margin=dict(r=150)
        )

        plot(fig)

plot_width_vs_fates(trials=1000, steps=30, materials=[(water, 1), (lead, 10), (graphite, 10)])

While the theoretical model assumes that neutron transmission through a material decreases exponentially with thickness, our simulation reveals a subtle deviation from this idealised behaviour. When plotting the number of transmitted particles as a function of material thickness, the curve fit deviates slightly from a perfect exponential, particularly at larger thicknesses. One possibility might be due to the physical structure of the simulation: neutrons start their path lengths at random distances inside the material (consistent with an exponential distribution), but particles that are reflected or absorbed before completing a full exponential step effectively truncate the distribution. As a result, the ensemble of transmission probabilities across many particles is not a pure exponential function of thickness. This effect is small but observable, and fitting only thicknesses around the attenuation length or thinner yields a closer match to the expected exponential decay and a more accurate attenuation length.

In [95]:
print_markdown(f"Execution time: ${time.time() - start:.2f}$ s")

Execution time: $7.73$ s


---

## 7. Simulating Transmission Through Adjacent Slabs Using the Woodcock Method

In this extension task, we apply the **Woodcock method** (also known as delta-tracking) to simulate neutron transmission through multiple adjacent slabs composed of different materials.

Neutrons are fired perpendicularly into the slabs at $x = 0$ and undergo stochastic random walks based on the Woodcock sampling method.

### Application of the Woodcock Method

First, we compare the **total macroscopic cross-sections** of the materials and identify the maximum value, $\Sigma_{\text{max}}$. This value is used for sampling step lengths throughout the entire simulation, ensuring uniform handling of different materials.

At each step:
- A random step length is drawn from an exponential distribution based on $\Sigma_{\text{max}}$.
- The neutron moves along a randomly generated isotropic direction.

After each step:
  - A uniform random number between 0 and 1 is generated.
  - If the random number is **less than** the **acceptance probability** (defined as $\Sigma_{\text{material}} / \Sigma_{\text{max}}$), the interaction is considered **real**.
    - In this case, we proceed to determine whether the neutron is absorbed, transmitted, or reflected.
  - If the random number is **greater than** the acceptance probability, the interaction is treated as a **virtual collision**.
    - No physical interaction (absorption or scattering) occurs.
    - The neutron simply continues with a new random step.

- If the neutron is in the most dense material in the supplied slabs (i.e., $\Sigma = \Sigma_{\text{max}}$), every interaction is automatically accepted without rejection sampling due to the probability of the collision being real `p_real` having the value 1.

Boundary conditions are handled as follows:
- If the neutron moves beyond $x > 2L$, it is considered **transmitted**.
- If it moves back through $x < 0$, it is considered **reflected**.
- If it undergoes a real collision inside the slab, absorption or scattering is determined based on the material properties.

This approach allows efficient and unbiased simulation of neutron transport through inhomogeneous materials with different cross-sections.

In [96]:
def random_walk_woodcock(slabs):
    x, y, z = (0, 0, 0)
    dx, dy, dz = (1, 0, 0)
    path = [(x, y, z)]
    max_cross_section = max(slab.material.total_cross_section for slab in slabs)
    total_width = sum(slab.width for slab in slabs)

    def get_slab_map():
        x_start = 0
        for slab in slabs:
            x_end = x_start + slab.width
            yield (x_start, x_end, slab.material, slab.material.total_cross_section / max_cross_section)
            x_start = x_end

    slab_positions = list(get_slab_map())

    def get_material_at(x):
        for start, end, material, p in slab_positions:
            if start <= x < end:
                return (material, p)

    while True:
        step_size = random_uniform_path_length(max_cross_section)
        x_new = x + step_size * dx
        y_new = y + step_size * dy
        z_new = z + step_size * dz
        x, y, z = (x_new, y_new, z_new)

        if x_new < 0:
            path.append((x,y,z))
            return "Reflected", path
        if x_new > total_width:
            path.append((x,y,z))
            return "Transmitted", path

        material, p_real = get_material_at(x_new)
        if material is None:
            raise ValueError("Position out of bounds")

        if np.random.uniform(0, 1) > p_real:
            continue
        else:
            if np.random.uniform(0, 1) < material.absorption_probability:
                path.append((x,y,z))
                return "Absorbed", path
            else:
                dx, dy, dz = generate_random_unit_vector()
                path.append((x,y,z))

In [97]:
def print_fate_rates(slabs, trials):
    counts = Counter([walk[0] for walk in [random_walk_woodcock(slabs) for _ in range(trials)]])

    markdown = "Fate | Count | Rate (%) \n"
    markdown += "--- | --- | --- \n"
    for fate, count in counts.items():
        rate = count / trials * 100
        markdown += f"{fate} | {count} | {rate:.2f} \n"
    markdown += "\n\n"
    print_markdown(markdown)

print_fate_rates([Slab(graphite, 10), Slab(lead, 10)], 3000)

Fate | Count | Rate (%) 
--- | --- | --- 
Reflected | 2318 | 77.27 
Transmitted | 448 | 14.93 
Absorbed | 234 | 7.80 





### Validation of the Woodcock Method Implementation

We will now run a series of tests to check that the data produced by the woodcock method produces statistically indistinguishable results when compared to eithe the simple method or an alternative woodcock. These tests taken together should provide confidence the logic is correct.

First we create a helper function `print_fate_rate_comparison` to produce these comparisons, given two tests.

In [98]:
def print_fate_rate_comparison(walk_func1, walk_func2, trials):
    fates1 = Counter([walk_func1()[0] for _ in range(trials)])
    fates2 = Counter([walk_func2()[0] for _ in range(trials)])

    all_fates = sorted(set(fates1) | set(fates2))

    markdown = (
        "| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |\n"
        "|------|---------------------------|---------------------------|--------|----------------|--------|\n"
    )

    for fate in all_fates:
        count1 = fates1.get(fate, 0)
        count2 = fates2.get(fate, 0)

        p1 = count1 / trials
        p2 = count2 / trials

        err1 = binomial_standard_error(p1, trials) * 100
        err2 = binomial_standard_error(p2, trials) * 100

        rate1 = p1 * 100
        rate2 = p2 * 100

        diff = abs(rate1 - rate2)
        combined_error = np.sqrt(err1**2 + err2**2)
        z_score = diff / combined_error if combined_error != 0 else float('inf')

        markdown += (
            f"{fate} | {rate1:.2f} ± {err1:.2f} | {rate2:.2f} ± {err2:.2f} | "
            f"{diff:.2f} | {combined_error:.2f} | {z_score:.2f} |\n"
        )

    print_markdown(markdown)

#### Split-Slab Equivalence Test

A slab of 5 cm of a material followed by another 5 cm of the same material should be statistically indistinguishable from a single 10 cm slab of that material.

This test compares Woodcock tracking on two 5 cm slabs against simple random walk on one 10 cm slab.

The results below validate that the Woodcock method does not introduce artefacts due to boundary crossings.

In [99]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(water, 3)]),
    lambda: random_walk(Slab(water, 6)),
    trials=1000
)

| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |
|------|---------------------------|---------------------------|--------|----------------|--------|
Absorbed | 19.60 ± 1.26 | 19.40 ± 1.25 | 0.20 | 1.77 | 0.11 |
Reflected | 78.30 ± 1.30 | 78.80 ± 1.29 | 0.50 | 1.84 | 0.27 |
Transmitted | 2.10 ± 0.45 | 1.80 ± 0.42 | 0.30 | 0.62 | 0.49 |


#### Split vs Single Slab (Same Method)

This test verifies that two consecutive 3 cm slabs of water produce statistically indistinguishable results from a single 6 cm slab, **using the same Woodcock tracking method** in both cases.

Since the material and total thickness are identical, any differences would indicate that the simulation is sensitive to artificial slab boundaries — which should not be the case.

The results below confirm that the Woodcock method maintains consistency across slab segmentation.


In [100]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(water, 3)]),
    lambda: random_walk_woodcock([Slab(water, 6)]),
    trials=1000
)

| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |
|------|---------------------------|---------------------------|--------|----------------|--------|
Absorbed | 17.10 ± 1.19 | 17.50 ± 1.20 | 0.40 | 1.69 | 0.24 |
Reflected | 80.90 ± 1.24 | 80.80 ± 1.25 | 0.10 | 1.76 | 0.06 |
Transmitted | 2.00 ± 0.44 | 1.70 ± 0.41 | 0.30 | 0.60 | 0.50 |


#### Woodcock vs simple (Same Geometry)

This test compares the fate statistics for a 10 cm water slab using both Woodcock and the simple random walk.

Identical geometry and material ensure that any difference arises purely from the tracking method itself. Close agreement validates the correctness of the Woodcock implementation.

In [101]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 6)]),
    lambda: random_walk(Slab(water, 6)),
    trials=1000
)

| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |
|------|---------------------------|---------------------------|--------|----------------|--------|
Absorbed | 19.60 ± 1.26 | 16.10 ± 1.16 | 3.50 | 1.71 | 2.05 |
Reflected | 78.00 ± 1.31 | 82.40 ± 1.20 | 4.40 | 1.78 | 2.47 |
Transmitted | 2.40 ± 0.48 | 1.50 ± 0.38 | 0.90 | 0.62 | 1.46 |


#### Vacuum Sandwich Test

This test validates that inserting a vacuum layer between two slabs of identical material does not alter the final outcome.

Physically, a vacuum introduces no scattering or absorption, and should not impact the overall transmission, reflection, or absorption rates.

The test compares 5 cm water + 5 cm vacuum + 5 cm water against a single 10 cm water slab using Woodcock tracking throughout.

In [102]:
vacuum = Material("Vacuum", 0, 0)

print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(vacuum, 5), Slab(water, 3)]),
    lambda: random_walk_woodcock([Slab(water, 6)]),
    trials=1000
)

| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |
|------|---------------------------|---------------------------|--------|----------------|--------|
Absorbed | 17.40 ± 1.20 | 17.50 ± 1.20 | 0.10 | 1.70 | 0.06 |
Reflected | 79.20 ± 1.28 | 80.20 ± 1.26 | 1.00 | 1.80 | 0.56 |
Transmitted | 3.40 ± 0.57 | 2.30 ± 0.47 | 1.10 | 0.74 | 1.48 |


#### Slab Order Independence

This test checks whether swapping the order of two slabs — graphite and lead — affects the results under Woodcock tracking.

Although the order does influence individual particle paths, the overall statistical fate rates (transmission, absorption, reflection) should remain the same in a large ensemble.

This test ensures that the Woodcock algorithm handles heterogeneous slab sequences without introducing order-dependent artefacts.

In [103]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(graphite, 5), Slab(lead, 5)]),
    lambda: random_walk_woodcock([Slab(lead, 5), Slab(graphite, 5)]),
    trials=1000
)

| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |
|------|---------------------------|---------------------------|--------|----------------|--------|
Absorbed | 4.30 ± 0.64 | 6.70 ± 0.79 | 2.40 | 1.02 | 2.36 |
Reflected | 65.20 ± 1.51 | 63.20 ± 1.53 | 2.00 | 2.14 | 0.93 |
Transmitted | 30.50 ± 1.46 | 30.10 ± 1.45 | 0.40 | 2.06 | 0.19 |


From the results, not every test was within 1 standard deviation, but we wouldn't expect them to be and increasing the number of trails, although computationally expensive, does show good agreement. These tests give us confidence that the implemented Woodcock method does have many qualities we would expect it to have, and gives consistent results in the cases we can test.

In [104]:
print_markdown(f"Execution time: ${time.time() - start:.2f}$ s")

Execution time: $13.99$ s